Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.core_expr_types.lambda_maps</a>
========

In [ ]:
import proveit
from proveit import Lambda, Conditional, Function
from proveit import c, f, g, h, x, y, F, G, fx, gx, Qx
from proveit.logic import Equals, Forall, Not
from proveit.numbers import one, two, three
%begin demonstrations

### Lambda substitution

In [ ]:
universal_eq = Forall(x, Equals(fx, gx))

In [ ]:
Lambda(x, fx).substitution(universal_eq, assumptions=[universal_eq])

In [ ]:
universal_eq_with_cond = Forall(x, Equals(fx, gx), condition=Qx)

In [ ]:
Lambda(x, Conditional(fx, Qx)).substitution(universal_eq_with_cond, 
                                            assumptions=[universal_eq_with_cond])

### Creating, Applying, & Checking Lambda Maps

#### Some example Lambda maps

In [ ]:
maps = { 1: Lambda(f, c),
         2: Lambda(x, fx),
         3: Lambda((f, g), Function(f, Function(g, c))),
         4: Lambda((f,g), Function(f, (g, Function(g, c)))),
         5: Lambda(g, Function(f, Function(g, c))),
         6: Lambda(f, Lambda(g, Function(f, Function(g, c)))),
         7: Lambda(g, Function(f, (g, Function(g, c)))),
         8: Lambda(f, Lambda(g, Function(f, (g, Function(g, c))))),
         9: Lambda((f, g, h), Function(h, (Function(f, c), Function(g, f)))),
        10: Lambda((f, g, h), Function(h, (Function(f, c), Function(g, f), g))),
        11: Lambda(x, Function(f, c)),
        12: Lambda(x, Function(f, f)),
        13: Lambda(x, Not(Function(f, f))),
        14: Lambda(f, Not(Function(f, f))),
        15: Lambda(f, Function(f, f)),
        16: Lambda((f, g, h), Function(h, (Function(f, g), Function(g, f))))
       };
for i in range(len(maps)):
    print(f'({i+1}) {maps[i+1]}')

#### We can apply our Lambda maps to appropriate inputs. Some examples:

In [ ]:
print(f'[{maps[1]}]({three}) = {maps[1].apply(three)}')
print(f'[{maps[2]}]({three}) = {maps[2].apply(three)}')
print(f'[{maps[3]}]({(F, G)}) = {maps[3].apply(F, G)}')
print(f'[{maps[4]}]({(F, G)}) = {maps[4].apply(F, G)}')

Should we be able to do the following (where I was expecting the <u>parameter</u> `g` to be distinguished from the <u>input</u> `g`)?<br/>
Should the Lambda application be handling the parameter `g` somehow to avoid variable capture?<br/>
This appears to be correct behavior, but also shows the (possible) importance then of checking resulting expressions for validity as well. The Lambda map looks fine, as does the input, but the result is a recursive expression that should be disallowed.

In [ ]:
print(f'[{maps[4]}]({(g, g)}) = {maps[4].apply(g, g)}')

#### Accessing pieces of a Lambda map.

In [ ]:
maps[4]

In [ ]:
# single-parameter Lambda can use .parameter, and
# multi-parameter Lambas can use .parameters, but more generally we have:
maps[4].parameter_or_parameters

For a Lambda map with multiple parameters, calling the (singular) `parameter` field yields an error:

In [ ]:
try:
    maps[4].parameter
except Exception as the_exception:
    print(f"{the_exception}")

But a Lambda map with a single parameter can use both the singular and plural field names, giving the parameter for the singular case, and a tuple containing the parameter for the plural case:

In [ ]:
maps[5].parameter

In [ ]:
maps[5].parameters

In [ ]:
maps[5].body

In [ ]:
# variables in the body?
maps[5].body.expr_info()

In [ ]:
maps[5].body._used_vars()

In [ ]:
maps[1].body._used_vars()

#### For Function bodies:

In [ ]:
if isinstance(maps[5].body, Function):
    print(f'Lambda body is a Function with')
    print(f'  Function operator:   {maps[5].body.operator}')
    print(f'  Function operand(s): {maps[5].body.operands}')

#### Infinite Order ?

In [ ]:
import math
math.inf

In [ ]:
1 + math.inf

### Operand/Operands and `_used_vars()` of Function()s

Function operations such as `Function(f, (x, y))`, giving $f(x,y)$, have no `operand` attribute, instead having an `operands` attribute.

In [ ]:
temp_expr = Function(f, Function(f, (x,y)))
temp_expr.operands

In fact, unary function operations such as $f(x)$ also have an `operands` (plural) attribute, wrapping the single operand in an ExprTuple:

In [ ]:
temp_expr = Function(f, x)
temp_expr.operands

In [ ]:
example_exprs = [x, f, Function(f,x), Function(f, f), Lambda(x, Function(f, x)), Lambda(g, Function(f, x))]
for _expr in example_exprs:
    print(f'{_expr._used_vars()}')

#### Related: Depth-First Search & The Discovery of Cycles

One approach to catching problematic dependencies among a collection of operands for a mult-operand function operation is to create and search through a graph or graph representation of the operator-operand relationships.</br>
In the graph below on the left, we might have had a function with various operands, including things like $A(B)$, $B(E)$, $E(F)$, etc., where $X(Y)$ is represented in the graph by a directed link from $X$ to $Y$. The graph on the right is the same but now without node $E$, which then removes the cycle.

![title](images/example_graph_with_cycle.png) ![title](images/example_graph_without_cycle.png)

Notice that if we cannot find a node with only outgoing links, we must have a cycle somewhere (i.e. such a node is necessary but not sufficient for the directed graph to be acyclic).

We might represent such graphs with adjacency dictionaries, like these:

In [ ]:
from proveit import A, B, C, D, E, F, G, H
example_graph_01 = {A: [B, F, H], B: [C, E], C: [D], D: [], E: [F], F: [B, G], G: [], H: []}
example_graph_02 = {A: [B, F, H], B: [C], C: [D], D: [], F: [B, G], G: [], H: []}
print(f'{example_graph_01.items()}')

#### Topological Sorting

(as described at https://www.geeksforgeeks.org/detect-cycle-in-a-graph/, with some minor modifications).

In [ ]:
from collections import deque

def is_cyclic(graph):
    '''
    Given a directed graph in the form of a dictionary of
    adjacencies, such as {A:[B, F, H], B:[C], ..., H:[]},
    use Kahn's algorithm for topological sorting to return
    True if and only if the graph contains a cycle.
    '''
    # get number of nodes
    num_nodes = len(graph.keys())
    
    # dict to store in-degree of each node
    in_degree = {x:0 for x in graph.keys()}
    # populate the in_degree dictionary
    for out_node in graph:
        for in_node in graph[out_node]:
            in_degree[in_node] += 1

    # a queue to store nodes with 0 in-degree
    q = deque()

    visited = 0 # count of visited vertices

    # Enqueue vertices with 0 in-degree
    for node in graph:
        if in_degree[node] == 0:
            q.append(node)

    # BFS traversal [?]
    while q:
        node = q.popleft()
        visited += 1

        # Reduce in-degree of adjacent vertices
        for node in graph[node]:
            in_degree[node] -= 1
            # If in-degree becomes 0, enqueue it
            if in_degree[node] == 0:
                q.append(node)
                
    # If not all vertices are visited (popped from q),
    # we have a cycle
    return visited != num_nodes


In [ ]:
example_graph_01.items()

In [ ]:
example_graph_01.keys()

In [ ]:
is_cyclic(example_graph_01)

In [ ]:
is_cyclic(example_graph_02)

#### The functions below are the most up-to-date versions of functions here that attempt to:

(1) canonicalize expressions to avoid variable crashing;

(2) construct a graphical representation of an expression (note this is not a syntax tree but a particular way of graphing expressions so we can detect cyclicality/recursion we want to disallow);

(3) generate a dictionary of minimum "orders" for an expression and its sub-expressions from a DAG of an expression (if the graph representation is in fact non-cyclic);

(4) return the minimum order of an expression.

Several of these implicitly or explicitly rely on the `is_cyclic()` function further above.

The functions still rely on a faulty `canonically_labeled()` function that will not return a full alpha-replacement for an expression containing multiple Lambda maps as sub-expressions, so that still needs work (as of Tues 2/18/25).

In [ ]:
########################################################################
import math
from proveit import ExprTuple, Literal, Operation, Variable

def canonicalize(expr):
    '''
    Given an Expression expr, parts of which might be Lambda
    expressions, build and return an equivalent Expression in which
    each Lambda expression component has been replaced by its
    canonical form. For example, the expression f(f -> f(x)) would
    be returned as f(_a -> _a(x)). This is not actually sufficient
    for some expressions, for example returning multiple Lambda
    maps in a single expression with the same dummy variables;
    thus this will be updated/upgraded soon.
    '''
    return expr.canonically_labeled()

########################################################################
def graph_from_expr(expr):
    '''
    Given an Operation (e.g. a Function) or Lambda map expr, generate
    and return an adjacency set dictionary representing the (directed)
    graph of dependencies among the operators and operands of the
    canonicalized form of expr.
    For example, the Function expression f(g(h), h(g)) would return
    the adjacency dictionary:
    {f(g(h), h(g)):{}, f:{g(h), h(g)}, g(h):{}, h(g):{}, g:{h}, h:{g}},
    where {} indicates an empty set (not an empty dictionary). The
    expr f([f -> f(x)]) would return the adjacency dictionary
    {  f([_a -> _a(x)]):{}, f:{[_a -> _a(x)]},
       [_a -> _a(x)]:{_a, _a{x}}, _a:{x}  }
    Notice that the original expr itself (in canonicalized form) is
    one of the nodes in the constructed graph, typically isolated from
    all others. For an expr that is neither an Operation nor a Lambda
    map, the adjacency dictionary returned is a trivial one of the
    form {expr: {}}.
    '''
    adj_dict = {}

    # convert to canonicalized form to avoid variable capture
    # (most relevant for exprs with Lambda maps).
    expr = expr.canonically_labeled()
    
    adj_dict[expr] = set() # will often be an isolated node
    if isinstance(expr, Operation):
        adj_dict[expr.operator] = set(expr.operands)
        for _operand in expr.operands:
            if _operand not in adj_dict:
                adj_dict[_operand] = set()
        for _op in expr.operands:
            _op_adj_dict = graph_from_expr(_op)
            for key in _op_adj_dict:
                if key in adj_dict:
                    # update previously established key by updating
                    # (not replacing) its set of adjacencies
                    adj_dict[key] = adj_dict[key].union(_op_adj_dict[key])
                else:
                    adj_dict[key] = _op_adj_dict[key]

########################################################################

    if isinstance(expr, Lambda):
        # For subsequent order computations, a Lambda map should be
        # linked to each of its parameters and to its body.
        for param in expr.parameters:
            adj_dict[expr].add(param)
            adj_dict[param] = set()
        adj_dict[expr].add(expr.body)
        # construct adjacency dictionary for Lambda map body
        body_adj_dict = graph_from_expr(expr.body)
        # update adjacency dictionary with body adjacency data
        for key in body_adj_dict:
            if key in adj_dict:
                # update previously-estab'd key:adj pair
                adj_dict[key] = adj_dict[key].union(body_adj_dict[key])
            else:
                adj_dict[key] = body_adj_dict[key]
            
    return adj_dict

########################################################################

def orders_from_dag(graph):
    '''
    Given a directed acyclic graph (DAG), in the form of an
    adjacency list dictionary, return a dictionary of node:order pairs
    corresponding to the nodes in the graph.
    For example, given the graph:
        { f(g(h), h(x)):{}, f:{g(h), h(x)}, g(h):{},
          h(x):{}, g:{h}, h:{x} }
    Return the order dictionary:
    {f(g(h), h(x)):0, f:1, g(h):0, h(x):0, g:2, h:1, x:0}.
    For convenience and flexibility, the supplied graph dict values
    can be lists or sets.
    '''
    # first a quick verification that the graph is acyclic
    # (otherwise, the results are nonsense)
    if is_cyclic(graph):
        raise ValueError(f"The graph {graph} submitted to " +
                         "'orders_from_graph()' appears to be cyclic. " +
                         "'orders_from_graph()' works only for directed " +
                         "acyclic graphs (DAGs).")
    
    # use dict/set comprehension to initialize order 0 for each node
    order_dict = {node:0 for node in graph}
    
    # collect all leaf nodes (nodes with out-degree 0)
    target_nodes = []
    for node in graph:
        if graph[node] == set() or graph[node] == []:
            target_nodes.append(node)
    
    # because the graph is a DAG, we know target_nodes is non-empty
    order = 1
    while target_nodes:
        connecting_nodes = []
        for target_node in target_nodes:
            for node in graph:
                if target_node in graph[node]:
                    connecting_nodes.append(node)
        for node in connecting_nodes:
            order_dict[node] = order # will auto-increase if needed
        target_nodes = connecting_nodes
        order += 1

    # then need to check if any of our isolated nodes are actually Lambda maps
    # (notice that nested Lambda maps will lead to some redundant processing here)
    for node in order_dict:
        if isinstance(node, Lambda):
            # print(f'Found a Lambda for updating: {node}')
            # more elaborate order calculation necessary
            # o(Lambda) = 1 + max(o(params), o(body)), where
            # o(params) = max(o(param_1), o(param_2), ..., o(param_n))
            params_order = 0 
            for param in node.parameters:
                params_order = max(params_order, order_dict[param])
            # print(f'params_order = {params_order}')
            order_dict[node] = 1 + max(params_order, order_dict[node.body])
            # print(f'order_dict[node] = {order_dict[node]}')

    # one last check: when expr is f(X),
    # where a non-zero order X now appears elsewhere - do we need to check that
    # o(f) > o(X), or does this happen automatically now? 
    # Actually it happens automatically based on the f -->-- X link in the graph.
    
    return order_dict

def min_order(expr):
    '''
    Return the minimum order of the expression expr,
    for an expr type Variable, Literal, Function,
    or Lambda map. Currently uses the min_order_helper()
    function, but can eventually be combined with that
    function to form a single function.
    '''
    _order_dict = min_order_helper(expr)
    if isinstance(_order_dict, dict):
        return _order_dict[expr]
    else:
        return _order_dict

def min_order_helper(expr):
    '''
    ########################################################################
    Given an Expression expr of type Variable, Literal, Function, or
    Lambda map, return a dictionary of values representing the order of
    the expression itself and the order of any operators and operand(s).
    For example, min_order_helper(f(f(x))) would eventually return the
    dictionary: {f(f(x)):0, f(x):0, f:1, , x:0}.
    min_order_helper is typically used indirectly through min_order().
    min_order_helper utilizes both the graph_from_exp() function and the
    is_cyclic() function.
    '''
    # construct an adjacency dictionary representation
    # of the graph of expr
    expr_graph = graph_from_expr(expr)

    # test the graph for being cyclic
    expr_graph_is_cyclic = is_cyclic(expr_graph)

    if not expr_graph_is_cyclic:
        # if expr graph is non-cyclic we should be fine
        expr_orders = orders_from_dag(expr_graph)
        # if isinstance(expr, Lambda):
        # # the order of a full Lambda map takes a little more work
        # # o(Lambda) = 1 + max(o(params), o(body)),
        # # where o(params) = max{o(param_1), o(param_2), ..., o(param_n)}
        #     params_order = -1
        #     for param in expr.parameters:
        #         params_order = max(params_order, expr_orders[param])
        #     expr_orders[expr] = 1 + max(params_order, expr_orders[expr.body])
        return expr_orders
    else:
        # if cyclic, we have a problem
        return {expr:math.inf}

    # still need to deal with lambda maps more generally
    # stuff below is left over from pervious incarnations
    
    
    _order_dict = {}       # an empty dictionary to build and store the order of each component
    _order_dict[expr] = 0  # include self with initial o(self) = 0

    # expr is a Label (i.e. a Variable or a Literal).
    if isinstance(expr, Variable) or isinstance(expr, Literal):
        # we are done; this is a "base case" with o(Label) = 0 when
        # no other information is available about the Label
        return _order_dict
    
    # if expr is a function operation of the form Function(operator, operand),
    if isinstance(expr, Function) and len(expr.operands)==1:
        _order_dict[expr.operator] = 0
        # the following might simply repeat the previous entry, and that's OK
        _order_dict[expr.operand] = 0
        operand_dict = min_order_helper(expr.operand)
        if not isinstance(operand_dict, dict):
            return operand_dict
        else:
            _order_dict[expr.operator] = 1 + operand_dict[expr.operand]
            if expr.operator in operand_dict and _order_dict[expr.operator] != operand_dict[expr.operator]:
                # operator depends on itself as an operand (somewhere)
                _order_dict[expr.operand]  = operand_dict[expr.operand]
                _order_dict[expr.operator] = math.inf
                _order_dict[expr]          = math.inf
                # do we need to merge?
            else:
                # update current order dict
                _order_dict.update(operand_dict)
                # if operator and operands have finite order, function operation has order 0
                # but if operator/operands have infinite order, fxn op also has inf order
                if _order_dict[expr.operand] == math.inf:
                    _order_dict[expr.operator] = math.inf
                if _order_dict[expr.operator] == math.inf:
                    _order_dict[expr] = math.inf
                return _order_dict

    # expr is a Lambda map (UNDER CONSTRUCTION)
    if isinstance(expr, Lambda):
        _body_dict = min_order_helper(expr.body)
        _params_dict = {}
        for _param in expr.parameters:
            if _param in _body_dict:
                _params_dict[_param] = _body_dict[_param]
            else:
                _params_dict[_param] = 0
        # print(f'_params_dict.values() = {_params_dict.values()}')
        _params_dict[expr.parameters] = max(_params_dict.values()) ## problem here
        _order_dict.update(_body_dict)
        _order_dict.update(_params_dict)
        _order_dict[expr] = 1 + max(_params_dict[expr.parameters], _body_dict[expr.body])
        return _order_dict

    # everything else right now
    else:
        for _var in expr._used_vars():
            _order_dict[_var] = 0
    return _order_dict


In [ ]:
from proveit import a, b, c, f, g, h, k, x
example_fxns = [
    Function(f, g), Function(f, Function(g, x)), Function(f, f),
    Function(f, Function(f, f)), Function(f, Function(f, Function(f, x))),
    Function(f, Function(g, f)), Function(g, Function(f, g)),
    Function(f, Function(g, Function(h, f))),
    Function(f, Function(g, Function(f, g))),
    Function(f, Function(g, Function(h, Function(x, two)))),
    Function(f, Function(g, Function(h, Function(f, two)))),
    Function(f, Function(g, Function(h, Function(x, g)))),
    Function(f, (Function(g, h), Function(h, g))),
    Function(f, (Function(g, h), Function(h, f))),
    Function(f, (Function(g, h), Function(x, y))),
    Function(f, (Function(a, b), Function(b, c), Function(c, a))),
    Function(f, Lambda(g, Function(h, g))),
    Function(g, Lambda(g, Function(h, g))),
    Function(f, (Lambda((a, b), Function(a, b)), Lambda((a, b), Function(b, a)), k))
];
for i, example in enumerate(example_fxns):
    num_str = f'({i+1})'
    print(f'{num_str:5} {example}')

In [ ]:
for i, fxn in enumerate(example_fxns):
    num_str = f'({i+1})'
    fxn_str = f'{fxn}'
    print(f'{num_str:4} {fxn_str:13}\n     has graph: {graph_from_expr(fxn)}\n')

In [ ]:
for i, fxn in enumerate(example_fxns):
    print(f'{(i+1)} {fxn} has order dict: {min_order_helper(fxn)}')

In [ ]:
from proveit import a, b, c, f, g, h, x
example_expressions = [
    Lambda(f, c),
    Lambda(f, Function(f, c)),
    Lambda((f, g), Function(f, Function(g, c))),
    Lambda((f, g), Function(f, (g, Function(g, c)))),
    Lambda(g, Function(f, Function(g, c))),
    Lambda(f, Lambda(g, Function(f, Function(g, c)))),
    Lambda(g, Function(f, (g, Function(g, c)))),
    Lambda(f, Lambda(g, Function(f, (g, Function(g, c))))),
    Lambda((f, g, h), Function(h, (Function(f, c), Function(g, f)))),
    Lambda((f, g, h), Function(h, (Function(f, c), Function(g, f), g))),
    Lambda(x, Function(f, c)),
    Lambda(x, Function(f, f)),
    Lambda(x, Not(Function(f, f))),
    Lambda(f, Not(Function(f, f))),
    Lambda(f, Function(f, f)),
    Lambda((f, g, h), Function(h, (Function(f, g), Function(g, f)))),
    Lambda((f, g), Function(f, Function(g, f))),
    Function(f, Lambda(g, h)),
    Function(f, Lambda(g, Function(h, g))),
    Function(f, Lambda(f, Function(f, x))),
    Function(f, Lambda(g, Function(g, f)))
];
for i, example in enumerate(example_expressions):
    num_str = f'({i+1})'
    print(f'{num_str:5} {example}')

In [ ]:
for i, example in enumerate(example_expressions):
    num_str = f'({i+1})'
    example_str = f'{example}'
    print(f'{num_str:5} {example_str:30} {min_order(example)}')

### Canonicalization

We can canonicalize an expression, converting all parameters to canonical forms. For example:

In [ ]:
display(example_expressions[5])
canonicalize(example_expressions[5])

In [ ]:
display(example_expressions[19])
canonicalize(example_expressions[19])

In [ ]:
display(example_expressions[20])
display(canonicalize(example_expressions[20]))
display(canonicalize(Function(f, x)))

In [ ]:
display(example_fxns[18])
display(canonicalize(example_fxns[18]))

Notice in the examples above that the non-parameter variables ($c$, $f$, and $x$) remain in the canonicalized versions. In fact, what the canonicalize() function is doing is working only on the parameters appearing in Lambda maps.

In [ ]:
display(example_expressions[15])
display(canonicalize(example_expressions[15]))

And note that an expression is equivalent to its canonicalized version, as we demonstrate for each of our example expressions:

In [ ]:
for i, example in enumerate(example_expressions):
    num_str = f'({i+1})'
    truth_value = example == canonicalize(example)
    print(f'{num_str:4} {example} == {canonicalize(example)}? {truth_value}')

### Some Practice and Reminders Working with sets

First of all, there is something odd about trying to create a python set containing Prove-It objects. As you'll see below, we can create sets of numbers and sets of strings, but the attempt to create a set of Variable objects produces a tuple-like structure (appearing with parentheses instead of braces):

In [ ]:
from proveit import a, b, c, d, f
example_set_01 = {a, b, c}

In [ ]:
from proveit import a, b, c, d, f
example_set_02 = {x.string() for x in [c, d, f]}

But print()'ing or display()'ing the tuple-like set gives the braces notation:

In [ ]:
print(example_set_01)

The `add()` method is mutating: this changes the set itself

In [ ]:
example_set_01.add(d)
display(example_set_01)

The union() method produces a new set, leaving the two united sets as they were:

In [ ]:
example_set_01.union(example_set_02)

In [ ]:
print(f"example_set_01 = {example_set_01}")
print(f"example_set_02 = {example_set_02}")

In [ ]:
example_set_04 = set()
example_set_04.add(b)
print(f'example_set_04 = {example_set_04}')

### Looking at the `canonically_labeled()` method

In [ ]:
ExprTuple(Lambda(f, Function(f, x)), Lambda(g, Function(g, x))).canonically_labeled()

In [ ]:
ExprTuple(Lambda(f, Function(f, x)), Lambda(x, Function(g, x))).canonically_labeled()

Notice the potential problem below, where we apply `canonicalize()` to an entire `ExprTuple`, and produce a list of canonicalized elements that end up sharing parameters:

In [ ]:
example_tuple = ExprTuple(
    Function(f, x), Lambda(f, Function(f, x)), Lambda(x, Function(g, x)),
    Function(h, Function(f, Function(g, h))), Function(h, h),
    Function(f, (Lambda((f, g),Function(f, g)), Lambda((f, g),Function(g, f)))))
display(example_tuple)
canonicalize(example_tuple)

In [ ]:
my_set = set()
for item in example_tuple:
    my_set.add(item.canonically_labeled())
print(f'my_set = {my_set}')

Notice that the function $f((f,g)\mapsto f(g), (f, g)\mapsto g(f))$ is labeled as cyclic, but shouldn't be. This is because the `canonically_labeled()` method is generating identical dummy variables for the two Lambda maps that serve as arguments for the function. Here is the result compared to other results that are perfectly fine:

In [ ]:
display(example_tuple)
for item in example_tuple:
    print(f'Cyclic? {is_cyclic(graph_from_expr(item))}')

And of course we can see the issue at the graph level, where the so-called graph isn't even a graph but just the ExprTuple expression itself, but the underlying `canonically_labeled()` call on the ExprTuple has generated the same canonical dummy variables for the Lambda maps inside the last function:

In [ ]:
graph_from_expr(example_tuple)

Not clear why this doesn't give an error, since the input doesn't appear to be a graph; need to check on this:

In [ ]:
is_cyclic(graph_from_expr(example_tuple))

### A potentially useful tool: `traverse_inner_expressions()`

In [ ]:
from proveit._core_.expression import free_vars, traverse_inner_expressions

The `traverse_inner_expressions()` function appears to traverse the syntax tree in a DFS fashion:

In [ ]:
for item in traverse_inner_expressions(example_tuple):
    display(item)

In [ ]:
from proveit._core_.expression import free_vars, traverse_inner_expressions
for item in example_tuple:
    display(free_vars(item))
for item in example_tuple:
    for inner_item in traverse_inner_expressions(item):
        display(inner_item, type(inner_item), inner_item.__class__)

### Some Work on Dummy Variables

In part, trying to make the cycling of the dummy variables somewhat more “regular” than the dummy-variable generation in Prove-It. The original algorithm has that line appending the `powers_of_26` with the last item times 3, which … isn't a power of 26, so at some point we end up having some irregularity in the procession of the dummy variables.

First, the original dummy variable generator, renamed as `dummy_var_original()`:

In [ ]:

def dummy_var_original(n):
    '''
    A replication of WW's dummy_var(n) method from var.py in _core_.
    Given an integer n, produce a "dummy" Variable that is the (n+1) element
    in the list: _X_, _Y_, _Z_, _XX_, _XY_, _XZ_, _YX_, _YY_, _YZ_, etc.

    Correction to description?
    Given a non-negative integer n (not clear what negative integers produce
    or if they even produce anything reliably), return the (n+1)st item
    in the list
    _a, _b, _c, ..., _z, _aa, _ab, _ac, ... _az, _ba, _bb, _bc, ..., _bz, ...
    That list is infinite and does not pre-exist.
    '''
    m = n
    powers_of_26 = [1, 26]  # for 26 letters in the alphabet
    # print(f'(1) powers_of_26 = {powers_of_26}')
    while m >= powers_of_26[-1]:
        m -= powers_of_26[-1]
        powers_of_26.append(powers_of_26[-1] * 3)
    # print(f'(2) powers_of_26 = {powers_of_26}')
    letters = ''
    powers_of_26.pop(-1)
    # print(f'(3) powers_of_26 = {powers_of_26}')
    while len(powers_of_26) > 0:
        pow_of_26 = powers_of_26.pop(-1)
        # print(f'(4) pow_of_26 = {pow_of_26}')
        k = int(m / pow_of_26)
        # print(f'(5) k = {k}')
        letters += chr(ord('a') + k)
        # print(f'(6) letters = {letters}')
        m -= k * pow_of_26
        # print(f'm = {m}')
    return Variable('_' + letters, latex_format=r'{_{-}' + letters + r'}')

For example, we have the following where we go from `_cz` to `_aaa` instead of from `_cz` to `_da` (see items (103) and (104)):

In [ ]:
for i in range(101, 111):
    print(f'({i}) {dummy_var_original(i)}')

So in some sense the algorithm is skipping ahead and hitting the 3-digit variables sooner than expected.

In the new dummy variable algorithm, we clean that up, but it's a bit complicated because the use of the modular arithmetic ends up treating the character 'a' as both a 0 sometimes and an incremental 1 sometimes. Hence we have an `expected_num_digits` calculation to help guide the formation of the variable strings:

In [ ]:
def dummy_var_new(n):
    '''
    For positive integer n, returns the nth dummy variable, using a
    simple base-26 conversion from non-negative (n-1) to variables of
    the form a, b, ..., z, aa, ab, ac, ..., az, aaa, aab, aac, etc.
    Algorithm somewhat complicated by the fact that we sometimes treat
    'a' as a 0 and sometimes like a '1' when dealing with the modular
    arithmetic and integer division. Without accounting for this,
    we end up with a sequence of variables like a, b, ..., z, ab, ...,
    where (for example) the aa element is skipped.
    '''
    digits = "abcdefghijklmnopqrstuvwxyz"
    n -= 1 # convert to 0 indexing; if asked for 1st value, want 0th in list
    result = ""

    m = n + 1
    expected_num_digits = 0
    while m > 0:
        expected_num_digits += 1
        m -= 26 ** expected_num_digits

    num_digits = 1
    while n >= 0 and num_digits <= expected_num_digits:
        remainder = n % 26
        result = digits[remainder] + result
        n = n // 26 - 1
        num_digits += 1
    
    return f'_{result}'

Now we see the continuation of the 2-digit variable names as expected:

In [ ]:
for i in range(1, 28):
    print(f'({i}) {dummy_var_new(i)}')

In [ ]:
for i in range(101, 107):
    print(f'({i}) {dummy_var_new(i)}')

and we shift from 2-digit to 3-digit as expected much later:

In [ ]:
for i in range(698, 708):
    print(f'({i}) {dummy_var_new(i)}')

and then from 3 to 4 digits in just the right way as well:

In [ ]:
for i in range(18276, 18285):
    print(f'({i}) {dummy_var_new(i)}')

and from 4 digits to 5 digits:

In [ ]:
for i in range(26 + 26**2 + 26**3 + 26**4 - 5, 26 + 26**2 + 26**3 + 26**4 + 5):
    print(f'({i}) {dummy_var_new(i)}')

In [ ]:
%end demonstrations